In [22]:
import numpy as np
from scipy.optimize import minimize
from math import sqrt
import pickle
import matplotlib.pyplot as plt

In [23]:
def initializeWeights(n_in, n_out):
    epsilon = sqrt(6) / sqrt(n_in + n_out + 1)
    W = (np.random.rand(n_out, n_in + 1) * 2 * epsilon) - epsilon
    return W


In [24]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def nnObjFunction(params, *args):
    n_input, n_hidden, n_class, training_data, training_label, lambdaval = args
    w1 = params[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
    w2 = params[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))
    num_samples = training_data.shape[0]

    # Forward propagation
    training_data = np.hstack((training_data, np.ones((num_samples, 1))))
    hidden_input = np.dot(training_data, w1.T)
    hidden_output = sigmoid(hidden_input)
    hidden_output = np.hstack((hidden_output, np.ones((hidden_output.shape[0], 1))))
    output_input = np.dot(hidden_output, w2.T)
    output = sigmoid(output_input)

    y = np.zeros((num_samples, n_class))
    y[np.arange(num_samples), training_label.astype(int)] = 1

    # Regularzition
    log_error = y * np.log(output) + (1 - y) * np.log(1 - output)
    obj_val = -np.sum(log_error) / num_samples
    obj_val += (lambdaval / (2 * num_samples)) * (np.sum(np.square(w1)) + np.sum(np.square(w2)))

    # Backpropagation
    delta_output = output - y
    grad_w2 = np.dot(delta_output.T, hidden_output) / num_samples + (lambdaval / num_samples) * w2
    delta_hidden = np.dot(delta_output, w2) * hidden_output * (1 - hidden_output)
    delta_hidden = delta_hidden[:, :-1]
    grad_w1 = np.dot(delta_hidden.T, training_data) / num_samples + (lambdaval / num_samples) * w1

    obj_grad = np.concatenate((grad_w1.flatten(), grad_w2.flatten()), 0)
    return obj_val, obj_grad

In [26]:
def nnPredict(w1, w2, data):
    data = np.hstack((data, np.ones((data.shape[0], 1))))
    hidden_input = np.dot(data, w1.T)
    hidden_output = sigmoid(hidden_input)
    hidden_output = np.hstack((hidden_output, np.ones((hidden_output.shape[0], 1))))
    output_input = np.dot(hidden_output, w2.T)
    output = sigmoid(output_input)
    labels = np.argmax(output, axis=1)
    return labels 

In [27]:
def preprocess():
    with open('face_all.pickle', 'rb') as f:
        pickle_obj = pickle.load(f)
    features = pickle_obj['Features']
    labels = pickle_obj['Labels']
    
    train_x = features[0:21100] / 255.0
    valid_x = features[21100:23765] / 255.0
    test_x = features[23765:] / 255.0

    train_y = labels[0][0:21100]
    valid_y = labels[0][21100:23765]
    test_y = labels[0][23765:]
    
    return train_x, train_y, valid_x, valid_y, test_x, test_y

In [ ]:
train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

#hyperparameters ----- Tried several values which i have metioned in the report 
n_input = train_data.shape[1]
n_hidden = 256  
n_class = 2
lambdaval = 55  

initial_w1 = initializeWeights(n_input, n_hidden)
initial_w2 = initializeWeights(n_hidden, n_class)
initialWeights = np.concatenate((initial_w1.flatten(), initial_w2.flatten()), 0)

args = (n_input, n_hidden, n_class, train_data, train_label, lambdaval)
opts = {'maxiter': 50}

nn_params = minimize(nnObjFunction, initialWeights, jac=True, args=args, method='CG', options=opts)
params = nn_params.x

# Reshaping back
w1 = params[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
w2 = params[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))


predicted_label = nnPredict(w1, w2, train_data)
print('\n Training set Accuracy: ' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

predicted_label = nnPredict(w1, w2, validation_data)
print('\n Validation set Accuracy: ' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

predicted_label = nnPredict(w1, w2, test_data)
print('\n Test set Accuracy: ' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')


 Training set Accuracy: 85.38388625592417%

 Validation set Accuracy: 84.16510318949342%

 Test set Accuracy: 85.99545798637396%
